In [2]:
from recurrentshop.cells import *
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Activation
from keras.layers import add, multiply, concatenate
from keras import backend as K
from keras.models import Sequential

In [3]:
class LSTMDecoderCell(ExtendedRNNCell):

    def __init__(self, hidden_dim=None, **kwargs):
        if hidden_dim:
            self.hidden_dim = hidden_dim
        else:
            self.hidden_dim = self.output_dim
        super(LSTMDecoderCell, self).__init__(**kwargs)

    def build_model(self, input_shape):
        hidden_dim = self.hidden_dim
        output_dim = self.output_dim

        x = Input(batch_shape=input_shape)
        h_tm1 = Input(batch_shape=(input_shape[0], hidden_dim))
        c_tm1 = Input(batch_shape=(input_shape[0], hidden_dim))

        W1 = Dense(hidden_dim * 4,
                   kernel_initializer=self.kernel_initializer,
                   kernel_regularizer=self.kernel_regularizer,
                   use_bias=False)
        W2 = Dense(output_dim,
                   kernel_initializer=self.kernel_initializer,
                   kernel_regularizer=self.kernel_regularizer,)
        U = Dense(hidden_dim * 4,
                  kernel_initializer=self.kernel_initializer,
                  kernel_regularizer=self.kernel_regularizer,)

        z = add([W1(x), U(h_tm1)])

        z0, z1, z2, z3 = get_slices(z, 4)
        i = Activation(self.recurrent_activation)(z0)
        f = Activation(self.recurrent_activation)(z1)
        c = add([multiply([f, c_tm1]), multiply([i, Activation(self.activation)(z2)])])
        o = Activation(self.recurrent_activation)(z3)
        h = multiply([o, Activation(self.activation)(c)])
        y = Activation(self.activation)(W2(h))

        return Model([x, h_tm1, c_tm1], [y, h, c])


class AttentionDecoderCell(ExtendedRNNCell):

    def __init__(self, hidden_dim=None, **kwargs):
        if hidden_dim:
            self.hidden_dim = hidden_dim
        else:
            self.hidden_dim = self.output_dim
        self.input_ndim = 3
        super(AttentionDecoderCell, self).__init__(**kwargs)


    def build_model(self, input_shape):
        
        input_dim = input_shape[-1]
        output_dim = self.output_dim
        input_length = input_shape[1]
        hidden_dim = self.hidden_dim

        x = Input(batch_shape=input_shape)
        h_tm1 = Input(batch_shape=(input_shape[0], hidden_dim))
        c_tm1 = Input(batch_shape=(input_shape[0], hidden_dim))
        
        W1 = Dense(hidden_dim * 4,
                   kernel_initializer=self.kernel_initializer,
                   kernel_regularizer=self.kernel_regularizer)
        W2 = Dense(output_dim,
                   kernel_initializer=self.kernel_initializer,
                   kernel_regularizer=self.kernel_regularizer)
        W3 = Dense(1,
                   kernel_initializer=self.kernel_initializer,
                   kernel_regularizer=self.kernel_regularizer)
        U = Dense(hidden_dim * 4,
                  kernel_initializer=self.kernel_initializer,
                  kernel_regularizer=self.kernel_regularizer)

        C = Lambda(lambda x: K.repeat(x, input_length), output_shape=(input_length, input_dim))(c_tm1)
        _xC = concatenate([x, C])
        _xC = Lambda(lambda x: K.reshape(x, (-1, input_dim + hidden_dim)), output_shape=(input_dim + hidden_dim,))(_xC)

        alpha = W3(_xC)
        alpha = Lambda(lambda x: K.reshape(x, (-1, input_length)), output_shape=(input_length,))(alpha)
        alpha = Activation('softmax')(alpha)

        _x = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=(1, 1)), output_shape=(input_dim,))([alpha, x])

        z = add([W1(_x), U(h_tm1)])

        z0, z1, z2, z3 = get_slices(z, 4)

        i = Activation(self.recurrent_activation)(z0)
        f = Activation(self.recurrent_activation)(z1)

        c = add([multiply([f, c_tm1]), multiply([i, Activation(self.activation)(z2)])])
        o = Activation(self.recurrent_activation)(z3)
        h = multiply([o, Activation(self.activation)(c)])
        y = Activation(self.activation)(W2(h))

        return Model([x, h_tm1, c_tm1], [y, h, c])

In [4]:
from __future__ import absolute_import
from recurrentshop import LSTMCell, RecurrentSequential 
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, TimeDistributed, Bidirectional, Input, Flatten
import os
import codecs
# 载入模型
from keras_bert import load_trained_model_from_checkpoint

In [5]:
# 定义地址
bert_model_path = "/Users/zhouwencheng/Desktop/Grass/data/model/ImportModel/BERT/chinese_L-12_H-768_A-12"
config_path = os.path.join(bert_model_path, 'bert_config.json')
checkpoint_path = os.path.join(bert_model_path, 'bert_model.ckpt')
vocab_path = os.path.join(bert_model_path, 'vocab.txt')

In [6]:
model_bert = load_trained_model_from_checkpoint(config_path, checkpoint_path)

W0812 01:30:02.673328 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 01:30:02.707922 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 01:30:02.811918 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 01:30:02.813101 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7

In [48]:
# model_bert.add(Dense(1000))
input_1 = Input(batch_shape=(None, 512))
model_output = model_bert([input_1, input_1]) 

In [64]:
input_2= Input(batch_shape=(None, 512))
model_2=Model(model_output)

# model = Sequential()
# model.add(model_2)

# model_decoder = Model(inputs, decoded) 

ValueError: Cannot add an empty model to a `Sequential` model.

In [63]:
model_2

In [49]:
input_1 = Input(batch_shape=(None, 512))
model = Sequential()
model.add(model_output)
input_1

TypeError: The added layer must be an instance of class Layer. Found: Tensor("model_2_7/Encoder-12-FeedForward-Norm/add_1:0", shape=(?, 512, 768), dtype=float32)

In [22]:
Model(model_decoder.inputs, model_decoder.outputs)
# model_decoder

In [24]:
input_1 = Input(batch_shape=(512,))
input_2 = Input(batch_shape=(512,None, None))

model_bert
model = Sequential()
model.add(Model(model_decoder.inputs, model_decoder.outputs))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_12 (Model)             (None, 128, 21100)        2942489   
Total params: 2,942,489
Trainable params: 2,942,489
Non-trainable params: 0
_________________________________________________________________


In [40]:
_input

<tf.Tensor 'input_67:0' shape=(?, 93, 300) dtype=float32>

In [42]:
output_dim=21100
output_length=128
batch_input_shape=None
batch_size=None
input_shape=None
input_length=512
input_dim=768
hidden_dim=128 
depth=1
bidirectional=True
unroll=False
stateful=False
dropout=0.0
shape=(None, 512, 768)

_input = Input(batch_shape=shape)
_input._keras_history[0].supports_masking = True
 

print("encoded.shape:", shape)
decoder = RecurrentSequential(decode=True, output_length=output_length,
                              unroll=unroll, stateful=stateful)
decoder.add(Dropout(dropout, batch_input_shape=(shape[0], shape[1], hidden_dim)))
decoder.add(AttentionDecoderCell(output_dim=output_dim, hidden_dim=hidden_dim))



print(model_bert.outputs[0].shape)
print(_input.shape)

inputs = [_input]
# _input = model_bert.outputs[0]

decoded = decoder(_input) 
# model_decoder = Model(inputs, decoded) 




# model = Sequential()
# model.add(model_decoder)
# model.add(Flatten())
# model.add(Dense(1000)) 

    
# model.summary()

encoded.shape: (None, 512, 768)
(?, 512, 768)
(?, 512, 768)


In [53]:
decoded

<tf.Tensor 'recurrent_sequential_7/transpose_1:0' shape=(?, ?, 21100) dtype=float32>

In [80]:
output_dim=300
output_length=5000
batch_input_shape=None
batch_size=None
input_shape=None
input_length=93
input_dim=300
hidden_dim=128 
depth=1
bidirectional=True
unroll=False
stateful=False
dropout=0.0

shape=(93, None, 300)

In [84]:
if isinstance(depth, int):
    depth = (depth, depth)
if batch_input_shape:
    shape = batch_input_shape
elif input_shape:
    shape = (batch_size,) + input_shape
elif input_dim:
    if input_length:
        shape = (batch_size,) + (input_length,) + (input_dim,)
    else:
        shape = (batch_size,) + (None,) + (input_dim,)
else:
    # TODO Proper error message
    raise TypeError
if hidden_dim is None:
    hidden_dim = output_dim

_input = Input(batch_shape=shape)
_input._keras_history[0].supports_masking = True

encoder = RecurrentSequential(unroll=unroll, stateful=stateful,
                              return_sequences=True)
encoder.add(LSTMCell(hidden_dim, batch_input_shape=(shape[0], shape[2])))

for _ in range(1, depth[0]):
    encoder.add(Dropout(dropout))
    encoder.add(LSTMCell(hidden_dim))

if bidirectional:
    encoder = Bidirectional(encoder, merge_mode='sum')
    encoder.forward_layer.build(shape)
    encoder.backward_layer.build(shape)
    # patch
    encoder.layer = encoder.forward_layer

encoded = encoder(_input)

print("encoded.shape:", shape)
decoder = RecurrentSequential(decode=True, output_length=output_length,
                              unroll=unroll, stateful=stateful)
decoder.add(Dropout(dropout, batch_input_shape=(shape[0], shape[1], hidden_dim)))
if depth[1] == 1:
    decoder.add(AttentionDecoderCell(output_dim=output_dim, hidden_dim=hidden_dim))
else:
    decoder.add(AttentionDecoderCell(output_dim=output_dim, hidden_dim=hidden_dim))
    for _ in range(depth[1] - 2):
        decoder.add(Dropout(dropout))
        decoder.add(LSTMDecoderCell(output_dim=hidden_dim, hidden_dim=hidden_dim))
    decoder.add(Dropout(dropout))
    decoder.add(LSTMDecoderCell(output_dim=output_dim, hidden_dim=hidden_dim))

inputs = [_input]
decoded = decoder(_input)
# decoded = decoder(encoded)
model = Model(inputs, decoded)
model.summary()

encoded.shape: (None, 93, 300)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_726 (InputLayer)          (None, 93, 300)      0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
recurrent_sequential_56 (Recurr (None, 5000, 300)    259289      input_726[0][0]                  
                                                                 private__opti